<h1>Exercise 3: The Resume Agent</h1>

### The Challenge
Let's build a sequential pipeline that feels like a real workflow.

**Goal:** Build an agent that takes raw user data and constructs a profile description step-by-step.

**Requirements:**
1. **State**: Needs `name` (str), `age` (int), `skills` (List[str]), and `result` (str).
2. **Node 1**: Start the `result` string with "[Name], welcome to the system! "
3. **Node 2**: Append "You are [Age] years old. "
4. **Node 3**: Append "You have skills in: [Skill1], [Skill2]..."
5. **Flow**: Connect them in order: `Node 1` -> `Node 2` -> `Node 3`.

### The Solution
Let's assemble our factory line.

### Step 1: Imports
We need `List` again from typing.

In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph

### Step 2: Define State
Structured data in, strings out.

In [ ]:
class AgentState(TypedDict):
    name: str           # User's name
    age: int            # User's age
    skills: List[str]   # List of skills e.g. ["Python", "AI"]
    result: str         # The final profile description

### Step 3: Define Nodes
We need three specialized workers.

In [ ]:
def firstNode(state: AgentState) -> AgentState:
    """ Node 1: Handles the Introduction """
    print("--- Starting Resume Generation ---")
    state["result"] = f"{state["name"]}, welcome to the system! "
    return state

In [ ]:
def secondNode(state: AgentState) -> AgentState:
    """ Node 2: Adds Age Information """ 
    # Append to the existing intro
    state["result"] = state["result"] + f"You are {state["age"]} years old ! "
    return state

In [ ]:
def thirdNode(state: AgentState) -> AgentState:
    """ Node 3: Lists the skills """ 
    state["result"] = state["result"] + "You have skills in: "
    
    # Standard Python loop to format the list nicely with commas and 'and'
    skills = state["skills"]
    for i, skill in enumerate(skills):
        if i == len(skills) - 1: # Last item
            state["result"] += f"and {skill}."
        else: 
            state["result"] += f"{skill}, "
            
    return state

### Step 4: Build the Pipeline
Chain them up! `1 -> 2 -> 3`.

In [ ]:
graph = StateGraph(AgentState)

# Add all nodes
graph.add_node("first", firstNode)
graph.add_node("second", secondNode)
graph.add_node("third", thirdNode)

# Define the Flow
graph.set_entry_point("first")
graph.add_edge("first", "second")  # 1 -> 2
graph.add_edge("second", "third") # 2 -> 3
graph.set_finish_point("third")   # 3 -> End

app = graph.compile()

### Step 5: Visualize
You should see a clean line of 3 nodes.

In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

### Step 6: Run It
Let's create a profile for Linda.

In [ ]:
# Define the full input state
input_data = {
    "name": "Linda", 
    "age": 31, 
    "skills": ["Python", "Machine Learning", "LangGraph"]
}

result = app.invoke(input_data)

### The Result
Check out the fully assembled string.

In [ ]:
print(result["result"])